## Assignment 1: Language Modeling

*This assignment is adapted from one created by David Gaddy, Daniel Fried, Nikita Kitaev, Mitchell Stern, Rodolfo Corona, John DeNero, and Dan Klein.*

In this assignment, you will implement several different types of language models for text.  We'll start with n-gram models, then move on to neural n-gram and transformer language models.

**Warning**: Do not start this assignment the day before it is due!  Some parts require 20 minutes or more to run, so debugging and tuning can take a significant amount of time.

Our dataset for this assignment will be the WikiText2 language modeling dataset.  This dataset comes with some of the basic preprocessing done for us, such as tokenization and rare word filtering (using the `<unk>` token).
Therefore, we can assume that all word types in the test set also appear at least once in the training set.
We'll also use the Huggingface `datasets` and `tokenizers` libraries to help with some of the data preprocessing, such as converting tokens into id numbers.

**Note on GPU usage**: You will need to use a GPU for the neural n-gram and transformer models but **not** for the n-gram models. Colab places some restrictions on GPU usage due to which you might get locked out after continuously using one (~8 hours). To avoid this, you should only use the GPU when needed, i.e., on training and inference for the last two parts of this assignment. You can enable / disable GPU usage by changing the Runtime type under the Runtime menu.
If you do get locked out of using a GPU, a potential workaround is to sign in using a different account.

When training a model on the GPU it is also a good idea to save your model periodically in case you get locked out. You can use `torch.save(network.state_dict(), path)` and `network.load_state_dict()` for this; see [here](https://pytorch.org/tutorials/recipes/recipes/saving_and_loading_models_for_inference.html).

**Grading rubric**
- 70% Results
    - 15% bigram_predictions.npy (correctness)
    - 15% trigram_backoff_predictions.npy (correctness)
    - 15% neural_trigram_predictions.npy (meets target)
    - 15% transformer_predictions.npy (meets target)
    - 10% transformer_predictions.npy (improvement over target)
- 30% Report
    - 25% Experimentation (Clarity, Correctness, and Analysis)
    - 5% Discussion of errors and Learning experience

In [ ]:
# Install some required packages.
!pip install transformers
!pip install datasets
!pip install torch==2.3.0 torchtext==0.18.0

In [ ]:
# This block handles some basic setup and data loading.
# You shouldn't need to edit this, but if you want to
# import other standard python packages, that is fine.

from collections import defaultdict, Counter
import numpy as np
import math
import tqdm
import random
import pdb

import torch
from torch import nn
import torch.nn.functional as F
import torchtext
import torch.optim as optim

# We'll use HuggingFace's datasets and tokenizers libraries, which are a bit
# heavy-duty for what we're doing, but it's worth getting to know them.

import transformers

from datasets import load_dataset
from tokenizers import Tokenizer
from tokenizers.models import WordLevel
from tokenizers.trainers import WordLevelTrainer
from tokenizers.pre_tokenizers import WhitespaceSplit

dataset = load_dataset("Salesforce/wikitext", "wikitext-2-v1")
tokenizer = Tokenizer(WordLevel(unk_token='<unk>'))
tokenizer.pre_tokenizer = WhitespaceSplit() # should be equivalent to split()

# "Training" a tokenizer below just feeds it all the tokens so it can map from
# word type to id.

trainer = WordLevelTrainer( # should only be 33,278 distinct types in Wikitext-2
    vocab_size=33300, special_tokens=["<unk>", "<eos>"])
generator_bsz = 512
all_splits_generator = (dataset[split][i:i+generator_bsz]["text"]
                        for split in ["train", "validation", "test"]
                          for i in range (0, len(dataset[split]), generator_bsz))
tokenizer.train_from_iterator(all_splits_generator, trainer)

# If desired, we could make a transformers tokenizer object now with:
# fast_tokenizer = PreTrainedTokenizerFast(tokenizer_object=tokenizer)

orig_vocab = tokenizer.get_vocab() # The tokenizer reserves a <pad> id, which we'll ignore.
word_types = sorted(list(orig_vocab.keys()), key=lambda w: orig_vocab[w]) # no <pad>
vocab = {w: i for i, w in enumerate(word_types)} # no <pad>
vocab_size = len(vocab)

# Make a single stream of tokens, with an <eos> after each newline.

train_text = []
for example in dataset["train"]["text"]:
  train_text.extend(tokenizer.encode(example).tokens + ["<eos>"])

validation_text = []
for example in dataset["validation"]["text"]:
  validation_text.extend(tokenizer.encode(example).tokens + ["<eos>"])

print(validation_text[:30])

We've implemented a unigram model here as a demonstration.

In [ ]:
class UnigramModel:
    def __init__(self, train_text):
        self.counts = Counter(train_text)
        self.total_count = len(train_text)

    def probability(self, word):
        return self.counts[word] / self.total_count

    def next_word_probabilities(self, text_prefix):
        """Return a list of probabilities for each word in the vocabulary."""
        return [self.probability(word) for word in word_types]

    def perplexity(self, full_text):
        """Return the perplexity of the model on a text as a float.

        full_text -- a list of string tokens
        """
        log_probabilities = []
        for word in full_text:
            # Note that the base of the log doesn't matter
            # as long as the log and exp use the same base.
            log_probabilities.append(math.log(self.probability(word), 2))
        return 2 ** -np.mean(log_probabilities)

unigram_demonstration_model = UnigramModel(train_text)
print('unigram validation perplexity:',
      unigram_demonstration_model.perplexity(validation_text))

def check_validity(model):
    """Performs several sanity checks on your model:
    1) That next_word_probabilities returns a valid distribution
    2) That perplexity matches a perplexity calculated from next_word_probabilities

    Although it is possible to calculate perplexity from next_word_probabilities,
    it is still good to have a separate more efficient method that only computes
    the probabilities of observed words.
    """

    log_probabilities = []
    for i in range(10):
        prefix = validation_text[:i]
        probs = model.next_word_probabilities(prefix)
        assert min(probs) >= 0, "Negative value in next_word_probabilities"
        assert max(probs) <= 1 + 1e-8, "Value larger than 1 in next_word_probabilities"
        assert abs(sum(probs)-1) < 1e-4, "next_word_probabilities do not sum to 1"

        word_id = vocab[validation_text[i]]
        selected_prob = probs[word_id]
        log_probabilities.append(math.log(selected_prob))

    perplexity = math.exp(-np.mean(log_probabilities))
    your_perplexity = model.perplexity(validation_text[:10])
    assert abs(perplexity-your_perplexity) < 0.1, "your perplexity does not " + \
    "match the one we calculated from `next_word_probabilities`,\n" + \
    "at least one of `perplexity` or `next_word_probabilities` is incorrect.\n" + \
    f"we calcuated {perplexity} from `next_word_probabilities`,\n" + \
    f"but your perplexity function returned {your_perplexity} (on a small sample)."


check_validity(unigram_demonstration_model)

To generate from a language model, we can sample one word at a time conditioning on the words we have generated so far.

In [ ]:
def generate_text(model, n=20, prefix=('<eos>', '<eos>')):
    prefix = list(prefix)
    for _ in range(n):
        probs = model.next_word_probabilities(prefix)
        word = random.choices(word_types, probs)[0]
        prefix.append(word)
    return ' '.join(prefix)

print(generate_text(unigram_demonstration_model))

In fact there are many strategies to get better-sounding samples, such as only sampling from the top-k words or sharpening the distribution with a temperature.  You can read more about sampling from a language model in this paper: https://arxiv.org/pdf/1904.09751.pdf.

You will need to submit some outputs from the models you implement for us to grade.  The following will download the required output files.

In [ ]:
!gdown --id 1QuBohoToduXKKJRCQAbkPl7Is73uGbdA
!gdown --id 1ZlXcG7FlG7YxjUMxqOD-KjtoKW2iTyuE

In [ ]:
def save_truncated_distribution(model, filename, short=True):
    """Generate a file of truncated distributions.

    Probability distributions over the full vocabulary are large,
    so we will truncate the distribution to a smaller vocabulary.

    Please do not edit this function
    """
    vocab_name = 'nu_eval_output_vocab'
    prefixes_name = 'nu_eval_prefixes'

    if short:
      vocab_name += '_short'
      prefixes_name += '_short'

    with open('{}.txt'.format(vocab_name), 'r') as eval_vocab_file:
        eval_vocab = [w.strip() for w in eval_vocab_file]
    eval_vocab_ids = [vocab[s] for s in eval_vocab]

    all_selected_probabilities = []
    with open('{}.txt'.format(prefixes_name), 'r') as eval_prefixes_file:
        lines = eval_prefixes_file.readlines()
        for line in tqdm.notebook.tqdm(lines, leave=False):
            prefix = line.strip().split(' ')
            probs = model.next_word_probabilities(prefix)
            selected_probs = np.array([probs[i] for i in eval_vocab_ids], dtype=np.float32)
            all_selected_probabilities.append(selected_probs)

    all_selected_probabilities = np.stack(all_selected_probabilities)
    np.save(filename, all_selected_probabilities)
    print('saved', filename)

In [ ]:
save_truncated_distribution(unigram_demonstration_model,
                            'unigram_demonstration_predictions.npy')

**Before you proceed**: At this point you should check whether you are able to upload the submission files to Gradescope. For this we will generate *dummy* prediction files by copying the unigram predictions above. Download the `unigrm_demonstration_predictions.npy` (you can do this by clicking the folder icon on left menu) and then copy this file and rename it to generate the required submision files:
* bigram_predictions.npy
* trigram_backoff_predictions.npy
* neural_trigram_predictions.npy
* transformer_predictions.npy

Also save a copy of this notebook as `assignment_1.ipynb` and create a `report.pdf` (this can be empty for now). Upload these files to Gradescope and confirm that the autograder runs and produces an output score of 0.

### N-gram Model

Now it's time to implement an n-gram language model.

Because not every n-gram will have been observed in training, use Lidstone smoothing to make sure no output word has probability 0.

$$P(w_2|w_1)=\frac{\#(w_1,w_2)+\alpha}{\#(w_1)+V\alpha}$$

where $V$ is the vocab size and $\#()$ is the count for the given bigram.  An alpha value around `3e-3`  should work.  Later, we'll replace this smoothing with model backoff.

One edge case you will need to handle is at the beginning of the text where you don't have `n-1` prior words.  You can handle this however you like as long as you produce a valid probability distribution, but just using a uniform distribution over the vocabulary is reasonable for the purposes of this assignment.

A properly implemented bi-gram model should get a perplexity below 505 on the validation set.

**Note**: Do not change the signature of the `next_word_probabilities` and `perplexity` functions.  We will use these as a common interface for all of the different model types.  Make sure these two functions call `n_gram_probability`, because later we are going to override `n_gram_probability` in a subclass.
Also, we suggest pre-computing and caching the counts $C$ when you initialize `NGramModel` for efficiency.

In [ ]:
class NGramModel:
    def __init__(self, train_text, n=2, alpha=3e-3):
        # get counts and perform any other setup
        self.n = n
        self.smoothing = alpha

        # YOUR CODE HERE

    def _to_string(self, tokens):
      return "_".join(tokens)

    def n_gram_probability(self, n_gram):
        """Return the probability of the last word in an n-gram.

        n_gram -- a list of string tokens
        returns the conditional probability of the last token given the rest.
        """
        assert len(n_gram) == self.n

        # YOUR CODE HERE

    def next_word_probabilities(self, text_prefix):
        """Return a list of probabilities for each word in the vocabulary."""

        # YOUR CODE HERE
        # use your function n_gram_probability
        # Recall word_types contains a list of words to return probabilities for

    def perplexity(self, full_text):
        """ full_text is a list of string tokens
        return perplexity as a float """

        # YOUR CODE HERE
        # use your function n_gram_probability
        # This method should differ a bit from the example unigram model because
        # the first n-1 words of full_text must be handled as a special case.

unigram_model = NGramModel(train_text, 1)
check_validity(unigram_model)
print('unigram validation perplexity:', unigram_model.perplexity(validation_text)) # this should be the almost the same as our unigram model perplexity above

bigram_model = NGramModel(train_text, n=2)
check_validity(bigram_model)
print('bigram validation perplexity:', bigram_model.perplexity(validation_text))

trigram_model = NGramModel(train_text, n=3)
check_validity(trigram_model)
print('trigram validation perplexity:', trigram_model.perplexity(validation_text)) # this won't do very well...

save_truncated_distribution(bigram_model, 'bigram_predictions.npy') # this might take a few minutes


Please download `bigram_predictions.npy` once you finish this section so that you can submit it.

In the block below, please report your bigram validation perplexity.

<!-- Do not remove this comment, it is used by the autograder: RqYJKsoTS6 -->

Bigram validation perplexity: ***fill in here***

We can also generate samples from the model to get an idea of how it is doing.

In [ ]:
print(generate_text(bigram_model))

We now free up some RAM, **it is important to run the cell below, otherwise you will likely run out of RAM in the Colab runtime.**

In [ ]:
# Free up some RAM.
del bigram_model
del trigram_model

This basic model works okay for bigrams, but a better strategy (especially for higher-order models) is to use backoff.  Implement backoff with absolute discounting.
$$P\left(w_i|h_i\right)=\frac{max\left\{\#(h_i, w_i)-d,0\right\}}{\#(h_i)} + \lambda(h_i) P(w_i|w_{i-n+2},\ldots, w_{i-1})$$

$$\lambda\left(h_i\right)=\frac{d N_{1+}(h_i)}{{\#(h_i)}}$$
where $h_i=(w_{i-n+1}, \ldots, w_{i-1})$ is the prefix before token $i$, $N_{1+}$ is the number of words that appear after the previous $n-1$ words (the number of times the max will select something other than 0 in the first equation).  If $\#(h_i)=0$, use the lower order model probability directly (the above equations would have a division by 0).

We found a discount $d$ of 0.9 to work well based on validation performance.  A trigram model with this discount value should get a validation perplexity below 272.

In [ ]:
class DiscountBackoffModel(NGramModel):
    def __init__(self, train_text, lower_order_model, n=2, delta=0.9):
        super().__init__(train_text, n=n)
        self.lower_order_model = lower_order_model
        self.discount = delta

        # YOUR CODE HERE

    def n_gram_probability(self, n_gram):
        assert len(n_gram) == self.n

        # YOUR CODE HERE
        # back off to the lower_order model with n'=n-1 using its n_gram_probability function

bigram_backoff_model = DiscountBackoffModel(train_text, unigram_model, 2)
trigram_backoff_model = DiscountBackoffModel(train_text, bigram_backoff_model, 3)
check_validity(trigram_backoff_model)
print('trigram backoff validation perplexity:', trigram_backoff_model.perplexity(validation_text))
save_truncated_distribution(trigram_backoff_model, 'trigram_backoff_predictions.npy') # this might take a few minutes

Free up RAM.

In [ ]:
# Release models we don't need any more.
del unigram_model
del bigram_backoff_model
del trigram_backoff_model

## Neural N-gram Model

In this section, you will implement the training and evaluation logic for a neural n-gram model.

**We have provided the model architecture for you in the `NeuralNGramNetwork` class.** It uses a simple feedforward neural network that takes the previous n-1 words and outputs a distribution over the next word.

Your task is to complete the `NeuralNGramModel` class. Specifically, you need to implement:
1.  **`train`**: The training loop (using the provided `NeuralNgramDataset` and `NeuralNGramNetwork`).
2.  **`next_word_probabilities`**: Logic to generate probabilities for a given context.
3.  **`perplexity`**: Evaluation logic to compute perplexity on a dataset.

The provided `NeuralNGramNetwork` follows this architecture (for your reference):
* Embed the words with dimension 128, then flatten into a single embedding for $n-1$ words.
* Run 2 hidden layers with 1024 hidden units, with ReLU activation and 0.1 dropout.
* Project down to size 128 before the final layer.
* Use weight tying for the embedding and final linear layer.

**Requirements for your implementation:**
* Train for 10 epochs with the Adam optimizer (should take around 15-20 minutes).
* Do early stopping based on validation set perplexity.
* A correct implementation should reach a validation perplexity below **226**.

In [ ]:
def ids(tokens):
    return [vocab[t] for t in tokens]

assert torch.cuda.is_available(), "no GPU found, in Colab go to 'Edit->Notebook settings' and choose a GPU hardware accelerator"

device = torch.device("cuda")

class NeuralNgramDataset(torch.utils.data.Dataset):
    """Iterates over ngrams in the input text data, returning the (n-1)gram
    prefix as well as the target token."""
    def __init__(self, text_token_ids, n):
        self.text_token_ids = text_token_ids
        self.n = n

    def __len__(self):
        return len(self.text_token_ids)

    def __getitem__(self, i):
        if i < self.n-1:
            prev_token_ids = [vocab['<eos>']] * (self.n-i-1) + self.text_token_ids[:i]
        else:
            prev_token_ids = self.text_token_ids[i-self.n+1:i]

        assert len(prev_token_ids) == self.n-1

        x = torch.tensor(prev_token_ids)
        y = torch.tensor(self.text_token_ids[i])
        return x, y

class NeuralNGramNetwork(nn.Module):
    # a PyTorch Module that holds the neural network for your model

    def __init__(self, n):
        super().__init__()
        self.n = n

        self.fc1 = nn.Linear((n-1) * 128, 1024)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(1024, 1024)
        self.fc3 = nn.Linear(1024, 128)
        self.out = nn.Linear(128, len(word_types))
        self.drop = nn.Dropout(p=0.1)

    def forward(self, x):
        # x is a tensor of inputs with shape (batch, n-1)
        # this function returns a tensor of log probabilities with shape (batch, vocab_size)

        word_vec = F.embedding(x, self.out.weight).view(x.size()[0], -1)
        # word_vec = self.embed(x) # (batch, n-1, 128)
        concat_vec = torch.reshape(word_vec, (word_vec.size()[0], -1)) # (batch, (n-1) * 128)
        h1 = self.drop(self.relu(self.fc1(concat_vec))) # (batch, 1024)
        h2 = self.drop(self.relu(self.fc2(h1))) # (batch, 1024)
        y = self.fc3(h2) # (batch, 128)
        return self.out(y) # (batch, len(word_types))


class NeuralNGramModel:
    # a class that wraps NeuralNGramNetwork to handle training and evaluation
    # it's ok if this doesn't work for unigram modeling
    def __init__(self, n):
        self.n = n
        self.network = NeuralNGramNetwork(n).to(device)

    def train(self):
        dataset = NeuralNgramDataset(ids(train_text), self.n)
        train_loader = torch.utils.data.DataLoader(dataset, batch_size=128, shuffle=True)
        # iterating over train_loader with a for loop will return a 2-tuple of batched tensors
        # the first tensor will be previous token ids with size (batch, n-1),
        # and the second will be the current token id with size (batch, )
        # you will need to move these tensors to GPU, e.g., by using the Tensor.cuda() function.

        # this will take some time to run; use tqdm.notebook.tqdm to get a progress bar
        # (see Assignment 0 for example)

        # The basic recipe for training networks will be the same as assignment 0.
        # A tutorial in Pytorch is also given here:
        # https://pytorch.org/tutorials/beginner/blitz/cifar10_tutorial.html#train-the-network

        # You should also print the perplexity on the validation set after each epoch by
        # calling self.perplexity(validation_text). You can do early stopping by
        # comparing this perplexity to the perplexity from the previous epoch
        # and stop training when it gets larger.

        # YOUR CODE HERE


    def next_word_probabilities(self, text_prefix):
        # you will need to convert text_prefix from strings to numbers with the `ids` function
        # if your `perplexity` function below is based on a NeuralNgramDataset DataLoader, you will need to use the same strategy for prefixes with less than n-1 tokens to pass the validity check
        #   the data loader appends extra "<eos>" (end of sentence) tokens to the start of the input so there are always enough to run the network
        # if the text_prefix is longer than n-1 tokens, make sure you truncate it before passing to the network

        # do a forward pass through the network and convert the log probabilities
        # into probabilities before returning.

        # YOUR CODE HERE
        # don't forget self.network.eval()
        # don't forget to move tensors to the GPU

    def perplexity(self, text):
        # you may want to use a DataLoader here with a NeuralNgramDataset
        # note that the NeuralNgramDataset prepends <eos> for the prefixes at the beginning less than n-1 tokens in length

        valdataset = NeuralNgramDataset(ids(text), self.n)
        val_loader = torch.utils.data.DataLoader(valdataset, batch_size=128, shuffle=False)

        # Iterate over the val_loader, do a forward pass across the batches and
        # compute the perplexities using the returned log probabilities.
        # (Hint: you can use torch.nn.functional.nll_loss for computing perplexity;
        # this will essentially select the log probability corresponding to the target word)

        # YOUR CODE HERE
        # don't forget self.network.eval()
        # don't forget to move tensors to the GPU

neural_trigram_model = NeuralNGramModel(3)
check_validity(neural_trigram_model)
neural_trigram_model.train()
print('neural trigram validation perplexity:', neural_trigram_model.perplexity(validation_text))

save_truncated_distribution(neural_trigram_model, 'neural_trigram_predictions.npy')

Fill in your neural trigram perplexity.

<!-- Do not remove this comment, it is used by the autograder: RqYJKsoTS6 -->

Neural trigram validation perplexity: ***fill in here***

Free up RAM.

In [ ]:
# Delete model we don't need.
del neural_trigram_model


## Transformer Model

For this stage of the project, you will implement Transformer language model.

We will divide the text into sequences of length `max_len=128` and batch them together. We will use a similar batching strategy as done for recurrent language modeling: sentences are concatenated together so that one sentence starts right after the other, and an unfinished sentence will be continued in the next batch. The `TransformerLMDataset` implemented below does that for you.

Recall that, for language modeling, we need to mask out the attention pattern such that every token only attends to preceding tokens and no other tokens which follow it. We will use the PyTorch `TransformerEncoder` module which implements this masking. You will need to provide a lower triangular mask of size `[max_len, max_len]` where all entries above the diagonal are -infinity and all entries on the diagonal or below are 0. Passing this to the `src_mask` argument of the `TransformerEncoder` will mask out the attention over future tokens (since the mask is *added* to the computed attention values).

You will also need to implement positional embeddings -- for the purpose of this assignment you can implement lookup style learned embeddings (one for each position up till `max_len`). These should be added to the word embeddings before passing to the `TransformerEncoder`. (You can also try implementing the sinusoidal positional embeddings from the [Attention is all you need](https://arxiv.org/abs/1706.03762) paper if you wish)

We expect your model to reach a validation perplexity below 135.  The following architecture and hyperparameters should be sufficient to get there.
* 6 Transformer layers with 8 attention heads each and `d_model=256` and `dim_feedforward=512`
* dropout of 0.3 within `TransformerEncoder`, after adding the positional embeddings, as well as at outputs of the `TransformerEncoder`
* learned positional embeddings initialized uniformly in [-0.01, 0.01]. You can create these using `nn.Parameter`
* instead of projecting directly from the Transformer output to the vocabulary size for softmax, project down to a smaller size first (e.g. 256->128->vocab_size).
* use the same weights for the embedding layer and the pre-softmax layer; dimension 128. Similar to projecting the output of the transformer, you will need to project the embeddings at the input to the size of the model first 128->256. You can define a `nn.Linear` layer for the output and use `F.embedding` using its `.weight` to embed the inputs.
* use a learning rate schedule which warms up the lr to a maximum value of 0.001 for the first 10% of steps, followed by linear decay back to 0 over the rest of the training. You can use `transformers.get_linear_schedule_with_warmup()` to implement this
* train with Adam (default hyperparameters) for at least 20 epochs using a batch size of 64
* use gradient clipping to limit the norm of the gradients to 1. You can use `torch.nn.utils.clip_grad_norm` just before you call `optimizer.step()`

We encourage you to try other architectures and hyperparameters, and you will likely find some that work better than the ones listed above. A proper implementation with these should be enough to receive full credit on the assignment, though.

In [ ]:
def ids(tokens):
    return [vocab[t] for t in tokens]


# YOU CAN DEFINE MORE HYPERPARAMETERS HERE
batch_size = 64
max_len = 128
num_layers = 6
hidden_size = 256
num_heads = 8
embed_size = 128
ff_size = 512
dropout = 0.3
clip = 1.
w_decay = 0.
lr = 0.001
n_epoch = 20

assert torch.cuda.is_available(), "no GPU found, in Colab go to 'Edit->Notebook settings' and choose a GPU hardware accelerator"

device = torch.device("cuda")

class TransformerLMDataset:
    """Returns a batch of sequences with their corresponding target word ids.
    Note that the returned tensors are of shapes `seq_len x batch_size`, hence
    the sequence length dimension appears before the batch dimension."""
    def __init__(self, text_token_ids, bsz, max_len=128):
        self.bsz = bsz
        self.max_len = max_len
        token_ids = torch.tensor(text_token_ids)
        ncontig = token_ids.size(0) // bsz
        token_ids = token_ids[:ncontig*bsz].view(bsz, -1) # bsz x ncontig
        self.token_ids = token_ids.t().contiguous() # ncontig x bsz

    def __len__(self):
        return int(math.ceil(self.token_ids.size(0) / self.max_len))

    def __iter__(self):
        for i in range(0, self.token_ids.size(0)-1, self.max_len):
            seqlen = min(self.max_len, self.token_ids.size(0) - i - 1)
            x = self.token_ids[i:i+seqlen] # max_len x bsz
            y = self.token_ids[i+1:i+seqlen+1] # max_len x bsz
            yield x, y

class TransformerNetwork(nn.Module):
    # a PyTorch Module that holds the neural network for your model

    def __init__(self, max_len=128, embed_size=128):  # feel free to add more parameters
        super().__init__()

        # Initialize the different layers needed in the computation graph.
        # A full list of available layers in Pytorch is given here:
        # https://pytorch.org/docs/stable/nn.html

        # In PyTorch, to construct a TransformerEncoder layer, we first need
        # to construct a nn.TransformerEncoderLayer and then pass it to
        # nn.TransformerEncoder.
        # Besides these you will need linear and dropout layers for this model.

        # Below we have initialized the positional embeddings for you.
        self.pos_embed = nn.Parameter(torch.FloatTensor(max_len, embed_size).uniform_(-0.01, 0.01))

        self.in_proj = nn.Linear(embed_size, hidden_size)
        self.pos_drop = nn.Dropout(p=dropout)
        self.enc_drop = nn.Dropout(p=dropout)

        encoder_layer = nn.TransformerEncoderLayer(
            d_model=hidden_size,
            nhead=num_heads,
            dim_feedforward=ff_size,
            dropout=dropout,
        )
        self.transformer = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)

        self.out_proj = nn.Linear(hidden_size, embed_size)
        self.out = nn.Linear(embed_size, len(word_types), bias=False)
        


    def _generate_mask(self, sz):
        """Return an attention mask with shape (sz, sz)."""

        # Note: you can use torch.triu (https://pytorch.org/docs/stable/generated/torch.triu.html) to build a upper triangular matrix.
        # For passing to the TransformerEncoder, we will need to replace 0s with -inf and 1s with 0

        mask = torch.triu(torch.ones(sz, sz), diagonal=1)
        mask = mask.masked_fill(mask == 1, float('-inf'))
        return mask


    def forward(self, x):
        """ x - a tensor with shape (seq_len, bsz)
            returns a tensor of log probabilities with shape (seq_len, bsz, len(word_types)).
        """

        # Make sure you add the positional embeddings initialized above to the word embeddings
        # extracted from the inputs.
        # Make sure you provide an attention mask to the TransformerEncoder (after moving it to the GPU).
        seq_len = x.size(0)
        word_vec = F.embedding(x, self.out.weight)
        pos = self.pos_embed[:seq_len].unsqueeze(1)
        word_vec = self.pos_drop(word_vec + pos)
        hidden = self.in_proj(word_vec)

        mask = self._generate_mask(seq_len).to(x.device)
        enc_out = self.transformer(hidden, mask)
        enc_out = self.enc_drop(enc_out)

        proj = self.out_proj(enc_out)
        logits = self.out(proj)
        return F.log_softmax(logits, dim=-1)


class TransformerModel:
    "A class that wraps TransformerNetwork to handle training and evaluation."

    def __init__(self):
        self.network = TransformerNetwork().cuda()

    def train(self):
        transformer_dataset = TransformerLMDataset(ids(train_text), batch_size, max_len)
        # Iterate over transformer_dataset with a for loop (optionally with tqdm).
        # Looping thru this dataset gives (x, y) tuples,
        # where x is a seqlen x batch_size token id tensor, and y is a seqlen x batch_size token id tensor.
        # The token ids in y are the next word targets for the sequence up till that position
        # in x.

        # The basic recipe for training the network will be the same as the NeuralNGramModel,
        # but note that the network will return the next word predictions of the entire
        # sequence together. You will need to reshape this in order to use the nll_loss.

        # Note: You will need a learning rate scheduler in addition to the optimizer.
        # Note: Initialize your optimizer and scheduler before starting the training loop.
        # Note: Make sure you update the parameters of your optimizer and scheduler after each training step.
        # Note: You should zero out previous gradients before each backpropragation:
        # https://pytorch.org/docs/stable/generated/torch.optim.Optimizer.zero_grad.html

        # You should also print the perplexity on the validation set after each epoch by
        # calling self.perplexity(validation_text). You can do early stopping by
        # comparing this perplexity to the perplexity from the previous epoch
        # and stop training when it gets larger.

        # You should also clip the gradients before performing an optimizer.step()
        # using torch.nn.utils.clip_grad_norm.

        optimizer = optim.Adam(self.network.parameters(), lr=lr, weight_decay=w_decay)
        total_steps = len(transformer_dataset) * n_epoch
        warmup_steps = max(1, int(0.1 * total_steps))
        scheduler = transformers.get_linear_schedule_with_warmup(
            optimizer, num_warmup_steps=warmup_steps, num_training_steps=total_steps
        )

        prev_ppl = None
        for epoch in range(n_epoch):
            self.network.train()
            for x, y in tqdm.notebook.tqdm(transformer_dataset, leave=False):
                x = x.cuda()
                y = y.cuda()

                optimizer.zero_grad()
                log_probs = self.network(x)
                loss = F.nll_loss(log_probs.view(-1, log_probs.size(-1)), y.reshape(-1))
                loss.backward()
                torch.nn.utils.clip_grad_norm(self.network.parameters(), clip)
                optimizer.step()
                scheduler.step()

            val_ppl = self.perplexity(validation_text)
            print(f'epoch {epoch+1} validation perplexity: {val_ppl}')
            if prev_ppl is not None and val_ppl > prev_ppl:
                break
            prev_ppl = val_ppl


    def next_word_probabilities(self, text_prefix):
        "Return a list of probabilities for each word in the vocabulary."

        # you will need to convert text_prefix from strings to numbers with the `ids` function
        # We won't be calling check_validity on the Transformer so you don't need
        # worry about the empty text_prefix.

        # Make sure you return probabilities instead of log probabilities!

        if len(text_prefix) == 0:
            prefix_ids = [vocab['<eos>']]
        else:
            prefix_ids = ids(text_prefix[-max_len:])

        x = torch.tensor(prefix_ids, dtype=torch.long).unsqueeze(1).cuda()
        self.network.eval()
        with torch.no_grad():
            log_probs = self.network(x)
            probs = torch.exp(log_probs[-1, 0]).detach().cpu().numpy()
        return probs.tolist()


    def perplexity(self, text):
        "Return perplexity as a float."
        # Use torch.no_grad() for extra speed.

        # Note that the nll_loss function, by default, computes the losses
        # averaged over each loss element in the batch.

        bsz = batch_size if len(text) > batch_size else 1
        transformer_dataset = TransformerLMDataset(ids(text), bsz, min(max_len, len(text)))

        # Loop over the transformer_dataset and get the outputs from your network.
        # Use nll_loss after reshaping these outputs and the corresponding targets
        # to get the average log probabilities.

        # Note that nll_loss by default will give you an average over all the elements
        # in a batch, keep track of the total number of elements for computing the
        # overall average for the perplexity.

        self.network.eval()
        total_loss = 0.0
        total_count = 0
        with torch.no_grad():
            for x, y in transformer_dataset:
                x = x.cuda()
                y = y.cuda()
                log_probs = self.network(x)
                loss = F.nll_loss(
                    log_probs.view(-1, log_probs.size(-1)),
                    y.reshape(-1),
                    reduction='sum'
                )
                total_loss += loss.item()
                total_count += y.numel()

        return math.exp(total_loss / total_count)


transformer_model = TransformerModel()
transformer_model.train()

print('transformer validation perplexity:', transformer_model.perplexity(validation_text))
save_truncated_distribution(transformer_model, 'transformer_predictions.npy')

# Experimentation: 2-Page Report

Now it's time for you to experiment.  Try to reach a validation perplexity below 120. You may either modify the transformer class above, or copy it to a new code cell and modify it there. Just **be sure to run the new code cell to generate results with your improved transformer**.  

It is okay if the bulk of your improvements are due to hyperparameter tuning (such as tuning the number or sizes of layers and attention heads, dropout, batch size, or the learning rate schedule). You are encouraged to explore the ideas below to further boost your performance. Here are some ideas:
* UID regularization - regularization based on the uniform information density (UID) hypothesis has shown improvement in language modeling. Check out https://arxiv.org/pdf/2105.07144.pdf for details.
* other types of regularization from https://arxiv.org/pdf/1708.02182.pdf, such as activation regularization, weight-drop regularization, embedding dropout. Note that these have been tested for LSTMs, so they may or may not work for transformers, but you can try them out.
* adaptive input representations (https://arxiv.org/pdf/1809.10853.pdf) which assign larger embeddings to frequent words
* ensembling - average the predictions of several models trained with different initialization random seeds

You may notice that most of these suggestions are regularization techniques.  This dataset is considered fairly small, so regularization is one of the best ways to improve performance.

For this section, you will submit a write-up describing the extensions and/or modifications that you tried.  Your write-up should be **2-page maximum** in length and should be submitted in PDF format. You may use any editor you like, but we recommend using LaTeX and working in an environment like Overleaf.

On the **first** page, please include the following for full credit (25 pts):
1.   A concise and precise description of the extension that you tried.
2.   A motivation for why you believed this approach might improve your model.
3.   A discussion of whether the extension was effective and/or an analysis of the results.  This will generally involve some combination of tables, learning curves, etc.
4.   A bottom-line summary of your results comparing validation perplexities of your improvement to the original transformer.

On the **second** page, please include the following for full credit (5 pts):
1. A discussion of errors and bugs you have encountered.
2. Your learning experiences from this implementation.

The **first page (25%)** will be assessed on the clarity, correctness, and interestingness of your experiment and analysis.

The **second page (5%)** will be graded based on completion: **a reasonable discussion of your errors and learning experience will be enough to collect full credit.**

The purpose of this exercise is to experiment, so feel free to try/ablate multiple of the suggestions above as well as any others you come up with! Even if your idea does not work, we would like to see a clear and concise description of what you tried and what were the results.
When you submit the file, please name it `report.pdf`.



### Submission

Upload a submission with the following files to Gradescope:
* assignment_1.ipynb (rename to match this exactly)
* transformer_predictions.npy (this should also include all improvements from your exploration)
* neural_trigram_predictions.npy
* trigram_backoff_predictions.npy
* bigram_predictions.npy
* report.pdf

You can upload files individually or as part of a zip file, but if using a zip file be sure you are zipping the files directly and not a folder that contains them.

Be sure to check the output of the autograder after it runs.  It should confirm that no files are missing and that the output files have the correct format.  Note that the test set perplexities shown by the autograder are on a different scale from your validation set perplexities due to selecting different text and truncating the distribution.  Don't worry if the values seem worse. We will compare your perplexity on the test set to our model's perplexity and assign a score based on that.